# Reactions - benchmarks

In [1]:
%matplotlib widget 
# global import
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
# local import
import lppydsmc as ld

## General theory

The idea here is to initialize a 0D box in which we inject ions that are going to neutralize by colliding to a selected plan.

In practice, an injection from the left is performed and neutralization occurs on the right boundary. The top and bottom boundaries are periodic ones.
The injection boundary is an "out" one : the particles that manage to go out this way simply leave the system.

An equilibrium should be reach. The ratio between I/I- depends on the neutralization probability.

An analytical solution is apparently also available.

## DSMC simulation now

In [2]:
path_reactions_1 = "/home/calot/Documents/projets/lppydsmc/benchmarks/config_files/reactions_1.ini"

In [4]:
# launching simulations
simulation = False
if(simulation):
    ld.main(path_reactions_1, save = True);

In [5]:
results_path = "/home/calot/Documents/projets/lppydsmc/benchmarks/results/reactions_2_1/monitoring.h5" # seems ok ...
store = pd.HDFStore(results_path)
print(store.keys())

['/dsmc_collisions', '/dsmc_tracking', '/fluxes', '/out_particles', '/particles', '/wall_collisions']


In [13]:
dt = 1e-7
particles_weight = 3200000000
volume_cell = 1e-9
nb_cells = 1
qty_to_density = particles_weight/(volume_cell*nb_cells)

In [14]:
df_particles = store['particles']

In [15]:
df_particles_I = df_particles.loc[df_particles['species']==0.0]
df_particles_Im = df_particles.loc[df_particles['species']==1.0]

In [16]:
evo_I = df_particles_I['x'].groupby(df_particles_I.index).count()
evo_Im = df_particles_Im['x'].groupby(df_particles_Im.index).count()

In [17]:
print(evo_I/evo_Im)

1       0.018519
2       0.009174
3       0.012346
4       0.018605
5       0.018587
          ...   
996     1.407080
997     1.405929
998     1.405891
999     1.400239
1000    1.396852
Name: x, Length: 1000, dtype: float64


In [18]:
fig, ax = plt.subplots(constrained_layout = True)
ax.plot(evo_Im.index*dt, evo_Im*qty_to_density, label = 'I-',  color ='b')
ax.plot(evo_I.index*dt, evo_I*qty_to_density, label = 'I', color ='r')
# ax.plot(times, save[:,0]/dV, '--', color ='b')
# ax.plot(times, save[:,1]/dV, '--', color ='r')

# ax.plot(evo_Im.index*dt, (evo_Im+evo_I)*qty_to_density, label = 'I+I-', color ='k')
# ax.plot(times, np.sum(save, axis =1)/dV, '--', color = 'k')

ax.legend(loc='best');
ax.set_xlabel('t (s)')
ax.set_ylabel(r'density ($m^{-3}$)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# DSMC study
df_dsmc_collisions = store['dsmc_collisions']
df_dsmc_collisions.head()

,cell_idx,idx_reactions,quantity
1,0.0,0.0,1.0
1,0.0,1.0,0.0
1,0.0,2.0,0.0
1,1.0,0.0,1.0
1,1.0,1.0,1.0


In [343]:
reaction_0 = df_dsmc_collisions.loc[df_dsmc_collisions['idx_reactions']== 0]

In [344]:
fig, ax = plt.subplots()
ax.plot(reaction_0.index, reaction_0['quantity'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [345]:
df_out = store['out_particles']

In [347]:
df_out_I = df_out.loc[df_out['species']==0.0]
df_out_Im = df_out.loc[df_out['species']==1.0]

In [352]:
count_I = df_out_I['x'].groupby(df_out_I.index).count()
count_Im = df_out_Im['x'].groupby(df_out_Im.index).count()

In [354]:
fig, ax = plt.subplots()
ax.plot(count_I.index, count_I, 'r')
ax.plot(count_Im.index, count_Im, 'b')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [355]:
df_wall_collisions = store['wall_collisions']

In [361]:
fig, ax = plt.subplots()
sns.scatterplot(data = df_wall_collisions, x = 'x', y = 'y', s = 0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>

In [367]:
df_partial = df_wall_collisions.loc[df_wall_collisions.index > 400]

In [368]:
df_partial.loc[df_partial['species']==1.0].groupby('reaction').count()

,x,y,species
reaction,,,
0.0,983,983,983
1.0,461,461,461


In [369]:
461/(983+461)

0.3192520775623269

In [366]:
df_wall_collisions

,x,y,species,reaction
1,2.233387e-05,0.000028,1.0,0.0
10,1.838758e-04,0.000995,1.0,0.0
10,9.690159e-05,0.000995,1.0,0.0
10,1.354459e-04,0.000990,1.0,0.0
20,4.043435e-04,0.000985,1.0,0.0
...,...,...,...,...
1000,6.125017e-05,0.000976,1.0,1.0
1000,9.304397e-04,0.000005,1.0,0.0
1000,7.637624e-06,0.000003,1.0,1.0
1000,2.366333e-04,0.000994,1.0,0.0


In [370]:
df_part_partial = df_particles.loc[df_particles.index > 400]

In [373]:
fig, ax = plt.subplots()
sns.histplot(data = df_part_partial.loc[df_part_partial['species']==0.0], x = 'x', y = 'y', ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>

In [376]:
fig, ax = plt.subplots()
sns.histplot(data = df_part_partial.loc[df_part_partial['species']==1.0], x = 'x', y = 'y', ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>

In [379]:
fig, ax = plt.subplots()
sns.histplot(data = df_part_partial.loc[df_part_partial['species']==0.0], x = 'vz', ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='vz', ylabel='Count'>